In [10]:
import numpy as np
import pandas as pd
from tabulate import tabulate
#1339 filas
df = pd.read_csv('insurance.csv')
df.columns = df.columns.str.strip()

physical_activity_levels = np.array(["sedentary", "moderate", "active"])
df['physical_activity'] = np.random.choice(physical_activity_levels, size = len(df))

print(tabulate(df, headers='keys', tablefmt='psql'))


"""
array_np = np.array(df2['value'])
resultado_np = np.sqrt(array_np)
df2["raiz_cuadrada"] = resultado_np
df2.to_csv("datos_modificados.csv", index = False)

#Filtrar filas donde la columna 'Value' tenga valores mayores que 1000
df2.loc[df2['value'] > 1000, 'value'] *=2 #df.loc[condición, "nombre_columna"]


#APLICA UN 10% A TODOS LOS VALORES DE LA COLUMNA RAÍZ CUADRADA Y GUARDA ESOS VALORES EN UNA NUEVA COLUMNA QUE SE VA A LLAMAR R.C BRUTA
#COGE TODOS LOS VALORES DE LA COLUMNA R.C BUTA QUE SEAN MENORES DE 60 Y MAYORES DE 40 Y LES VAMOS A ELEVAR AL CUADRADO
array = np.array(df2['raiz_cuadrada']) 
porcentaje = array * 0.10
resultado = array - porcentaje
df2['rc_bruta'] = resultado
df2.to_csv("porcentaje.csv", index = False)

df2.loc[df2['rc_bruta'] < 60 and df2['rc_bruta'] > 40, 'rc_bruta'] **2
df2.to_csv("resultado.csv", index = False)
print(df2.head()) 
"""


+------+-------+--------+-------+------------+----------+-----------+------------+---------------------+
|      |   age | sex    |   bmi |   children | smoker   | region    |   expenses | physycal_activity   |
|------+-------+--------+-------+------------+----------+-----------+------------+---------------------|
|    0 |    19 | female |  27.9 |          0 | yes      | southwest |   16884.9  | moderate            |
|    1 |    18 | male   |  33.8 |          1 | no       | southeast |    1725.55 | active              |
|    2 |    28 | male   |  33   |          3 | no       | southeast |    4449.46 | active              |
|    3 |    33 | male   |  22.7 |          0 | no       | northwest |   21984.5  | moderate            |
|    4 |    32 | male   |  28.9 |          0 | no       | northwest |    3866.86 | moderate            |
|    5 |    31 | female |  25.7 |          0 | no       | southeast |    3756.62 | active              |
|    6 |    46 | female |  33.4 |          1 | no      

'\narray_np = np.array(df2[\'value\'])\nresultado_np = np.sqrt(array_np)\ndf2["raiz_cuadrada"] = resultado_np\ndf2.to_csv("datos_modificados.csv", index = False)\n\n#Filtrar filas donde la columna \'Value\' tenga valores mayores que 1000\ndf2.loc[df2[\'value\'] > 1000, \'value\'] *=2 #df.loc[condición, "nombre_columna"]\n\n\n#APLICA UN 10% A TODOS LOS VALORES DE LA COLUMNA RAÍZ CUADRADA Y GUARDA ESOS VALORES EN UNA NUEVA COLUMNA QUE SE VA A LLAMAR R.C BRUTA\n#COGE TODOS LOS VALORES DE LA COLUMNA R.C BUTA QUE SEAN MENORES DE 60 Y MAYORES DE 40 Y LES VAMOS A ELEVAR AL CUADRADO\narray = np.array(df2[\'raiz_cuadrada\']) \nporcentaje = array * 0.10\nresultado = array - porcentaje\ndf2[\'rc_bruta\'] = resultado\ndf2.to_csv("porcentaje.csv", index = False)\n\ndf2.loc[df2[\'rc_bruta\'] < 60 and df2[\'rc_bruta\'] > 40, \'rc_bruta\'] **2\ndf2.to_csv("resultado.csv", index = False)\nprint(df2.head()) \n'